## Qs
- Do we have year, to split train / test by year? 
- what did we do for patents with multiple documents ?
- word-embeddings (train our own or use pre-trained ones)

In [309]:
# import packages 

import tensorflow as tf
import pandas as pd  
import numpy as np 
import gzip
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Input, Embedding, Reshape, Flatten, Conv1D, Conv2D, MaxPool2D, GlobalMaxPool1D, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras import utils
from keras import optimizers, models

from sklearn import model_selection, preprocessing

import matplotlib.pyplot as plt
%matplotlib inline

In [87]:
# load data 

# features 
current_dir = %pwd
abstract = pd.read_table(current_dir+'/out_zipped/docNumberToAbsText.txt.gz',compression='gzip',
                         sep='|', names = ['doc_num', 'abstract'], header=None)
claim = pd.read_table(current_dir+'/out_zipped/docNumberToClaimText.txt.gz',compression='gzip',
                         sep='|', names = ['doc_num', 'claim'], header=None)
desc = pd.read_table(current_dir+'/out_zipped/docNumberToDescText.txt.gz',compression='gzip',
                         sep='|', names = ['doc_num', 'desc'], header=None)
title = pd.read_table(current_dir+'/out_zipped/docNumberToInvTitle.txt.gz',compression='gzip',
                         sep='|', names = ['doc_num', 'title'], header=None)
file_name = pd.read_table(current_dir+'/out_zipped/fileNameToDocNumber.txt.gz',compression='gzip',
                         sep='|', names = ['file_name', 'doc_num'], header=None)

# labels 
labels = pd.read_table(current_dir+'/out_zipped/docNumberToLabelSubClass.txt.gz',compression='gzip', header=None)
labels = labels[0].str.split('|', expand=True).rename(columns={0:'doc_num'})
clas = pd.read_table(current_dir+'/out_zipped/docNumberToClassText.txt.gz',compression='gzip', header=None)
clas = clas[0].str.split('|', expand=True).rename(columns={0:'doc_num'})


In [163]:
# joining the datasets using outer join, NOT removing the patents that are missing sections
# DISCUSSION: should we change it to inner join (i.e. removing the patents that are missing sections)
X = pd.concat([title.set_index('doc_num'), 
           abstract.set_index('doc_num'), 
           claim.set_index('doc_num'), 
           desc.set_index('doc_num')], axis=1).sort_index() #  join='inner'

Y = labels.set_index('doc_num').sort_index().set_index(X.index) # fixing the index mismatch
Y.columns=["label{}".format(i) for i in range(1,15)] # renaming columns 

print(X.shape)
print(Y.shape)

(25000, 4)
(25000, 14)


In [199]:
# data cleaning 
assert Y['label1'].isnull().sum() == 0 # there is no document with 

print("{} documents with null title".format(X['title'].isnull().sum()))
print("{} documents with null claims".format(X['claim'].isnull().sum()))
print("{} documents with null abstract".format(X['abstract'].isnull().sum()))
print("{} documents with null description".format(X['desc'].isnull().sum()))

# remove documents with null sections (title and abstract)
X_clean = X.dropna(how='any')
null_idx = X[~X.index.isin(X_clean.index)].index # storing the removed indices (i.e. document numbers)
assert X.shape[0] - null_idx.shape[0] == X_clean.shape[0] # making sure the row counts match

# removing the documents with null sections from the labels as well 
Y_clean = Y.loc[X_clean.index]

# some checks
assert X_clean.shape[0] == Y_clean.shape[0]
assert ((Y_clean.index == X_clean.index)*1).sum() == X_clean.shape[0]

# lower casing everything
X_clean = X_clean.apply(lambda x: x.str.lower())
Y_clean = Y_clean.apply(lambda x: x.str.lower())


1835 documents with null title
0 documents with null claims
58 documents with null abstract
0 documents with null description


In [201]:
# creating data set 
# in this first pass we will concat the text of all patent sections 
# for labels we will use the first label only 

data = pd.DataFrame()
data['full_text'] = X_clean['title'] + " " + X_clean['claim'] + " " + X_clean['abstract'] + " " + X_clean['desc']
data['label1'] = Y_clean['label1']

#data.full_text[100005]

In [231]:
data

,full_text,label1
doc_num,,
100005,image downsampling using redundant pixel remov...,"image data processing or generation, in general"
100008,electric power variation compensating device a...,wind motors
100009,die for tubular film extrusion a die for the e...,shaping or joining of plastics; shaping of sub...
100010,stencil printer a stencil printer comprising: ...,"apparatus or devices for manifolding, duplicat..."
100018,system containing a plurality of drams and a b...,electric digital data processing
100023,a method of manufacturing coverings and a cove...,shaping or joining of plastics; shaping of sub...
100027,high-frequency switch a high-frequency switch ...,transmission
100028,a dual mode drum brake device a dual mode drum...,couplings for transmitting rotation
100029,information recording medium an information re...,materials for applications not otherwise provi...


In [265]:
# examing the document length 
doc_lenghts = data.full_text.str.len()
print("Average document length: {} words".format(round(np.mean(doc_lenghts))))
print("Median document length: {} words".format(round(np.median(doc_lenghts))))
print("Max document length: {} words".format(round(np.max(doc_lenghts))))
print("Min document length: {} words".format(round(np.min(doc_lenghts))))

Average document length: 2560 words
Median document length: 2580.0 words
Max document length: 126735 words
Min document length: 1056 words


### Train Test Split

In [271]:
# train test split 
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(data['full_text'], data['label1'], test_size=0.2)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(18486,)
(4622,)
(18486,)
(4622,)


### Encoding Labels

In [316]:
# encode the target variable 
encoder = preprocessing.LabelEncoder()
Y_train = encoder.fit_transform(Y_train)
Y_test = encoder.fit_transform(Y_test)

#pd.Series(Y_train).value_counts(sort=True)

### Word Embeddings

In [294]:
# sequence length is set to median document length
sequence_length = 2580
embedding_dim = 300

In [230]:
# word-embeddings: representing documents using a dense vector representation
# Word embeddings can be trained using the input corpus itself or 
# can be generated using pre-trained word embeddings such as Glove, FastText, and Word2Vec

# step 1. Loading the pretrained word embeddings
embeddings_index = {}
for i, line in enumerate(open('pretrained_word_embeddings/wiki-news-300d-1M.vec')):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

In [266]:
# step 2. Creating a tokenizer object using Keras preprocessing object 
# the tokenizer has a default filter that removes all punctuation, plus tabs and line breaks, minus the ' character.
token = text.Tokenizer()
token.fit_on_texts(data['full_text'])
word_index = token.word_index

In [286]:
print('Number of words in our vocabulary: {}'.format(len(word_index.keys())))

Number of words in our vocabulary: 71451


In [267]:
# step 3. Transforming text documents to sequence of tokens and padding them to ensure equal length vectors
# choosing the median document length as max length for padding 
X_train_seq = sequence.pad_sequences(token.texts_to_sequences(X_train), maxlen=sequence_length)
X_test_seq = sequence.pad_sequences(token.texts_to_sequences(X_test), maxlen=sequence_length)

In [273]:
X_train_seq.shape

(18486, 2580)

In [275]:
# step 4. Creating a mapping of tokens and their respective embeddings
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [278]:
embedding_matrix.shape

(71452, 300)

In [288]:
X_train_seq.shape

(18486, 2580)

### CNN

In [337]:
# Define Parameters 
batch_size = 32
epochs = 2

vocabulary_size = len(word_index) + 1
sequence_length = sequence_length
embedding_dim = embedding_dim
num_filters = 100
kernel_size = 3
units = pd.Series(Y_train).value_counts().shape[0]

In [338]:
# Input Layer
input_layer = Input(shape=(sequence_length,), dtype='int32')

# Word embedding Layer
embedding_layer = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, input_length=sequence_length, 
                            weights=[embedding_matrix], trainable=False)(input_layer)
embedding_layer = SpatialDropout1D(0.3)(embedding_layer)

# Convolutional Layer
conv_layer = Conv1D(num_filters, kernel_size=kernel_size, activation="relu")(embedding_layer)

# Pooling Layer
pooling_layer = GlobalMaxPool1D()(conv_layer)

# Output Layers
output_layer1 = Dense(50, activation="relu")(pooling_layer)
output_layer1 = Dropout(0.25)(output_layer1)
output_layer2 = Dense(units= units, activation="softmax")(output_layer1)

# Compile the model
model = models.Model(inputs=input_layer, outputs=output_layer2,)
model.compile(optimizer=optimizers.Adam(lr=1e-3), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 2580)              0         
_________________________________________________________________
embedding_10 (Embedding)     (None, 2580, 300)         21435600  
_________________________________________________________________
spatial_dropout1d_9 (Spatial (None, 2580, 300)         0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 2578, 100)         90100     
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_6 (Dropout)          (None, 50)                0         
__________

In [336]:
pd.Series(Y_train).value_counts().shape[0]

532

In [339]:
from keras.utils import to_categorical
y_binary = to_categorical(Y_train)
y_binary.shape

(18486, 532)

In [ ]:
# Training 
history = model.fit(X_train_seq, y_binary,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)


Train on 16637 samples, validate on 1849 samples
Epoch 1/2
10720/16637 [==================>...........] - ETA: 2:43 - loss: 5.3612 - acc: 0.0527

## Model 

In [ ]:
# define parameters
batch_size = 32
epochs = 2

In [ ]:
# simple fully connected layer NN

model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])


In [ ]:
# fit